<a href="https://colab.research.google.com/github/fansie1/chemRxnClassification/blob/main/chemRxnBetter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install evaluate
!pip install wandb
!pip install sacremoses
!pip install spacy
!pip install -U --no-cache-dir gdown --pre
!pip install seqeval

import pdb
import wandb
use_wandb = False
if use_wandb:
  wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.6/188.6 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!gdown --no-cookies https://drive.google.com/uc?id=1--UFHJY3ldytNTsLcseIMDXEyzFeYEuj
!gdown --no-cookies https://drive.google.com/uc?id=1-oYW7JgHyhFj1HpbYvpHdsK8y-RwF2pW
!gdown --no-cookies https://drive.google.com/uc?id=1-KkDBrlyr3DF91BOTgp-vgCLwMe2JAmX

In [ ]:
# 获取数据集
# train = "/content/drive/MyDrive/chemrxnconfig/prod/train_optim_concat.txt"
# dev = "/content/drive/MyDrive/chemrxnconfig/prod/dev_optim_concat.txt"
# test = "/content/drive/MyDrive/chemrxnconfig/prod/test_optim_concat.txt"
train = "/content/train_optim_concat.txt"
dev = "/content/dev_optim_concat.txt"
test = "/content/test_optim_concat.txt"

bert_tag_label_id = -100
spacy_tag_label_id = -100
pad_label_id = -100
pad_token = 0
prod_labels = ["O", "B-Prod", "I-Prod"]

label2id = {context: i for i, context in enumerate(prod_labels)}
id2label = {i: context for i, context in enumerate(prod_labels)}
def read_examples_from_file(file_path):
    guid_index = 1
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words, labels = [], []
        metainfo = None
        for line in f:
            line = line.rstrip()
            if line.startswith("#\tpassage"):
                metainfo = line
            elif line == "":
                if words:
                    examples.append({
                            "guid":f"{guid_index}",
                            "words":words,
                            "metainfo":metainfo,
                            "labels":labels
                        })
                    guid_index += 1
                    words, labels = [], []
            else:
                splits = line.split("\t")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1])
                else:
                    # Examples could have no label for plain test files
                    labels.append("O")
        if words:

            examples.append({
                    "guid":f"{guid_index}",
                    "words":words,
                    "metainfo":metainfo,
                    "labels":labels
                 })

    return examples

train_input = read_examples_from_file(train)
dev_input = read_examples_from_file(dev)
test_input = read_examples_from_file(test)

In [ ]:
# tokenize
from transformers import AutoTokenizer, AutoConfig
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
spacy_model = spacy.load("en_core_web_sm")
en = English()

checkpoint = "jiangg/chembert_cased"
# checkpoint = "dmis-lab/biobert-large-cased-v1.1-squad"
# checkpoint = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
# checkpoint = "microsoft/biogpt"
# spacy_pos_token = {
#     "ADJ": "[unused1]",
#     "ADP": "[unused2]",
#     "ADV": "[unused3]",
#     "AUX": "[unused4]",
#     "CCONJ": "[unused5]",
#     "DET": "[unused6]",
#     "INTJ": "[unused7]",
#     "NOUN": "[unused8]",
#     "NUM": "[unused9]",
#     "PART": "[unused10]",
#     "PRON": "[unused11]",
#     "PROPN": "[unused12]",
#     "PUNCT": "[unused13]",
#     "SCONJ": "[unused14]",
#     "SYM": "[unused15]",
#     "VERB": "[unused16]",
#     "X": "[unused17]",
# }

# just useing the  noun and verb
spacy_pos_token = {"NOUN": "[unused8]", "VERB": "[unused16]",}

CLS = "[CLS]"
SEP = "[SEP]"
max_sentence = 256
tokenizer = AutoTokenizer.from_pretrained(checkpoint)



def build_spacy(sent_list):
  return {
      "spacy": spacy_model(' '.join(sent_list)),
      "find_idx": build_sent_idx(sent_list)
}


def build_sent_idx(sent_list):
  total_len = 0
  find_idx = []
  for i, word in enumerate(sent_list):
    find_idx.append(str(total_len) + "-" + str(total_len + len(word) - 1))
    # stridx2listidx[str(total_len) + "-" + str(total_len + len(word))] = i
    total_len = total_len + len(word) + 1
  return find_idx

def do_find_idx(find_idx, idx):
  for i, func in enumerate(find_idx):
    func_list = func.split("-")
    if (int(func_list[0]) <= idx and int(func_list[1]) >= idx):
      return i


def check_append_word(tokens_list, word):
  if (len(tokens_list) + 1) > max_sentence - 1:
    return
  tokens_list.append(word)

In [ ]:
# dataloader
from datasets import Dataset, DatasetDict

def tokenize_function(example):
  word_list = example["words"]
  original_labels = example["labels"]
  spacy_result = build_spacy(word_list)
  spacy = spacy_result["spacy"]
  sent_idx = spacy_result["find_idx"]
  tokens = [CLS]
  labels = [bert_tag_label_id]
  last_idx = 0
  sub_token_tag = None
  for _, word in enumerate(spacy):
    original_idx = do_find_idx(sent_idx, word.idx)
    # the sub token from same token
    if original_idx == last_idx:
      if not sub_token_tag and spacy_pos_token.get(word.pos_):
        sub_token_tag = spacy_pos_token.get(word.pos_)
    # the sub token from new token
    else:
      last_token = word_list[last_idx]
      first_sub_token = True
      for sub_token in tokenizer.tokenize(last_token):
        check_append_word(tokens, sub_token)
        label_tag = original_labels[last_idx] if first_sub_token else "O"
        check_append_word(labels,  label2id[label_tag])
        first_sub_token = False
      # add verb or noun token
      if sub_token_tag:
        check_append_word(tokens, sub_token_tag)
        check_append_word(labels, spacy_tag_label_id)
      last_idx = original_idx
  assert len(tokens) == len(labels)
  input_ids, token_type_ids, attention_mask, labels = padding_fn(
      tokenizer.convert_tokens_to_ids(tokens),
      [0] * len(tokens),
      [1] * len(tokens),
      labels,
  )
  decoder_mask = [(x != pad_label_id) for x in labels]
  assert len(input_ids) == max_sentence
  assert len(token_type_ids) == max_sentence
  assert len(attention_mask) == max_sentence
  assert len(labels) == max_sentence
  assert len(decoder_mask) == max_sentence
  return {
    "input_ids": input_ids,
    "token_type_ids": token_type_ids,
    "attention_mask": attention_mask,
    "labels": labels,
    "decoder_mask": decoder_mask
  }

def padding_fn(input_ids, token_type_ids, attention_mask, labels):
  pad_len = max_sentence - len(input_ids)
  input_ids += [pad_token] * pad_len
  token_type_ids += [0] * pad_len
  attention_mask += [0] * pad_len
  labels += [pad_label_id] * pad_len
  return input_ids, token_type_ids, attention_mask, labels


config = AutoConfig.from_pretrained(
      checkpoint,
      num_labels=3,
      id2label=id2label,
      label2id=label2id,
)
hidden_size=768
num_train_epochs=20

train_dataset = Dataset.from_list(train_input)
dev_dataset = Dataset.from_list(dev_input)
test_dataset = Dataset.from_list(test_input)

datasetDict = DatasetDict({
    "train": train_dataset,
    "validation": dev_dataset,
    "test": test_dataset
})


tokenized_datasets = datasetDict.map(tokenize_function)
tokenized_datasets = tokenized_datasets.remove_columns(["words", "guid", "metainfo"])
tokenized_datasets.set_format("torch")

In [ ]:
from transformers import DataCollatorWithPadding, DataCollatorForTokenClassification, default_data_collator
from torch.utils.data import DataLoader
batch_size = 16
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="pt")
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True,  batch_size=batch_size, collate_fn=default_data_collator)
eval_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=batch_size, collate_fn=default_data_collator)
test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=batch_size, collate_fn=default_data_collator)

In [ ]:
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
from transformers import BertForTokenClassification, TrainingArguments, AutoConfig



class BertForTagging(BertForTokenClassification):
    def __init__(self, config):
        super(BertForTagging, self).__init__(config)
        self.classifier = nn.Linear(hidden_size, config.num_labels)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        outputs = (logits,)

        loss_fct = CrossEntropyLoss()
        active_loss = attention_mask.view(-1) == 1
        active_logits = logits.view(-1, self.num_labels)
        active_labels = torch.where(
            active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
        )
        loss = loss_fct(active_logits, active_labels)
        outputs = (loss,) + outputs

        return outputs  # (loss), scores, (hidden_states), (attentions)

    def decode(self, logits, mask):
        preds = torch.argmax(logits, dim=2).cpu().numpy()
        batch_size, seq_len = preds.shape
        preds_list = [[] for _ in range(batch_size)]
        for i in range(batch_size):
            for j in range(seq_len):
                if mask[i, j]:
                    preds_list[i].append(preds[i,j])
        return preds_list

    def decode_labels(self, labels, mask):
        labels = labels.cpu().numpy()
        batch_size, seq_len = labels.shape
        labels_list = [[] for _ in range(batch_size)]
        for i in range(batch_size):
          for j in range(seq_len):
            if mask[i, j]:
              labels_list[i].append(labels[i, j])
        return labels_list



In [ ]:
# train and validation
from sklearn import metrics
from seqeval.metrics import f1_score, precision_score, recall_score
from tqdm import tqdm
import numpy as np
from typing import Dict
import wandb

use_wandb = False
if use_wandb:
  wandb.login()


if use_wandb:
  wandb.init(project="rxn_better_train")



def compute_metrics(predictions, label_ids):
    label_list_str = [[id2label[x] for x in seq] for seq in label_ids]
    preds_list_str = [[id2label[x] for x in seq] for seq in predictions]
    preds_list = [j for i in predictions for j in i]
    label_list = [j for i in label_ids for j in i]
    # pdb.set_trace()
    return {
        "precision": metrics.accuracy_score(label_list, preds_list),
        # "precision": precision_score(label_list, preds_list),
        "recall": recall_score(label_list_str, preds_list_str),
        "f1": f1_score(label_list_str, preds_list_str),
    }

def train_fn(data_loader, model, optimizer, scheduler, device, accumulation_steps=1):
    model.train()
    total_loss = 0
    total_accuracy = 0
    # pdb.set_trace()
    with tqdm(enumerate(data_loader), unit="batch", total=len(data_loader)) as tepoch:
        for batch_index, dataset in tepoch:
            tepoch.set_description(f"Epoch Started")

            input_ids = dataset['input_ids']
            attention_mask = dataset['attention_mask']
            token_type_ids = dataset['token_type_ids']
            labels = dataset['labels']

            input_ids = input_ids.to(device, dtype = torch.long)
            attention_mask = attention_mask.to(device, dtype = torch.long)
            token_type_ids = token_type_ids.to(device, dtype = torch.long)
            labels = labels.to(device, dtype = torch.long)

            optimizer.zero_grad()

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
                )
            loss = outputs[0]
            loss.backward()
            decoder_mask = dataset['decoder_mask']
            decode_predictions = model.decode(outputs[1], decoder_mask)
            decode_labels = model.decode_labels(labels, decoder_mask)
            metrics_result = compute_metrics(decode_predictions, decode_labels)
            # pdb.set_trace()
            train_accuracy = 100.0 * metrics_result["precision"]
            # train_accuracy = 100.0 * calculate_accuracy(outputs[1], labels, attention_mask)
            tepoch.set_postfix(loss=loss.item(), accuracy=train_accuracy)
            if use_wandb:
              wandb.log({"train_accuracy": train_accuracy, "train_loss": loss})
            if (batch_index+1) % accumulation_steps == 0 :
                optimizer.step()
                scheduler.step()

            total_loss += loss.item()
            total_accuracy += train_accuracy

    return total_loss/len(data_loader), total_accuracy/len(data_loader)


def eval_fn(data_loader, model, device):
    model.eval()

    final_labels = []
    final_pred = []
    loss = 0
    with torch.no_grad():
        for _, dataset in tqdm(enumerate(data_loader), total=len(data_loader)):
            input_ids = dataset['input_ids']
            attention_mask = dataset['attention_mask']
            token_type_ids = dataset['token_type_ids']
            labels = dataset['labels']
            decoder_mask = dataset['decoder_mask']

            input_ids = input_ids.to(device, dtype = torch.long)
            attention_mask = attention_mask.to(device, dtype = torch.long)
            token_type_ids = token_type_ids.to(device, dtype = torch.long)
            labels = labels.to(device, dtype = torch.long)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )

            loss = outputs[0]
            loss += loss.item()
            decode_predictions = model.decode(outputs[1], decoder_mask)
            decode_labels = model.decode_labels(labels, decoder_mask)
            final_labels.extend(decode_labels)
            final_pred.extend(decode_predictions)

    return loss/len(data_loader), final_pred, final_labels


def calculate_accuracy(outputs, labels, mask, type_acr = 'batch'):
    if type_acr == 'batch':
        # 从GPU中脱离出来并且返回一个全新的不带梯度的tensor，然后转成numpy
        mask = mask.cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()[mask == 1]
        outputs = outputs.cpu().detach().numpy()[mask == 1]
        # 返回参数最大的下表  axis=a 表示以a维来统计
        predictions = np.argmax(outputs, axis=1)
        return metrics.accuracy_score(labels.tolist(), predictions.tolist())
    return metrics.accuracy_score(labels, outputs)

In [ ]:
from transformers import get_linear_schedule_with_warmup, AdamW
from evaluate import evaluator

model = BertForTagging(config)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-4},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0},
]

num_training_steps = int(len(train_dataloader)/batch_size * num_train_epochs)

optimizer = AdamW(optimizer_parameters, lr=1e-3)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps
)

def validate_model(model, dataloader):
  model.eval()
  # metrics = evaluate.combine(["f1", "precision", "recall"])
  metrics = evaluator("token-classification")
  for batch in dataloader:
    batch = {k:v.to(device) for k,v in batch.items()}
    with torch.no_grad():
      output = model(**batch)
    logits = output[1]
    predictions = torch.argmax(logits, dim=-1)
    labels = batch["labels"]
    ids = batch["input_ids"]
    assert len(predictions) == len(labels)
    # for pred, lab, token in zip(predictions, labels, ids):
    #   if (pred != lab):
    #     write_txt("/result.txt", tokenizer.decode(token), pred, lab)
    # pdb.set_trace()
    metrics.add_batch(predictions=predictions.view(-1), references=labels.view(-1))
  return metrics.compute()

for e in range(num_train_epochs):
  print("Epoch:", e)
  loss, train_accuracy = train_fn(train_dataloader, model, optimizer, scheduler, device)
  print(f"Total Epoch Train Accuracy : {train_accuracy} with loss : {loss}")
  loss, predicted, labels = eval_fn(eval_dataloader, model, device)
  result = compute_metrics(predicted, labels)
  print(f"Total Epoch Eval Accuracy : {100.0 *  result['precision']}, Recall : {100.0 * result['recall']}, f1 : {100.0 * result['f1']}")
  # print("----------validate----------")
  # validate_model(model, eval_dataloader)
  # print("----------test----------")
  # validate_model(model, test_dataloader)

In [ ]:


# validate_model(model, eval_dataloader)